In [1]:
import numpy as np
import pandas as pd

In [2]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_curve, roc_auc_score

In [3]:
# берем датасет о пассажирах Титаника (https://www.kaggle.com/c/titanic/data) 
# с дальнейшей классификацией - выжил пассажир или нет (столбец Survived)
# использовать будем только файл train.csv, файл test.csv - не будем

df = pd.read_csv("train.csv", index_col="PassengerId")

df.head()

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 891 entries, 1 to 891
Data columns (total 11 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Pclass    891 non-null    int64  
 2   Name      891 non-null    object 
 3   Sex       891 non-null    object 
 4   Age       714 non-null    float64
 5   SibSp     891 non-null    int64  
 6   Parch     891 non-null    int64  
 7   Ticket    891 non-null    object 
 8   Fare      891 non-null    float64
 9   Cabin     204 non-null    object 
 10  Embarked  889 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 83.5+ KB


In [5]:
# удаляем столбца, которые содержат ненужные (на первый взгляд!) данные для классификации 

df.drop(["Cabin", "Name", "Ticket"], axis=1, inplace=True)

In [6]:
# заполняем пропущенные значения возраста (Age) средним

df["Age"].fillna(df["Age"].mean(), inplace=True)

In [7]:
df["Embarked"].value_counts()

S    644
C    168
Q     77
Name: Embarked, dtype: int64

In [8]:
# заполняем пропущенные значения Embarked наиболее часто встречающимся значением (модой)

df["Embarked"].fillna("S", inplace=True)

In [9]:
# кодируем пол целочисленными значениями

df["Sex"] = df["Sex"].map({"female": 1, "male": 0})

In [10]:
# делаем столбец "класс пассажира" категориальным и кодируем категориальные признаки ("Pclass", "Embarked") dummy-переменными

df["Pclass"] = df["Pclass"].astype("category")
df = pd.get_dummies(df)

df.head()

,Survived,Sex,Age,SibSp,Parch,Fare,Pclass_1,Pclass_2,Pclass_3,Embarked_C,Embarked_Q,Embarked_S
PassengerId,,,,,,,,,,,,
1,0,0,22.0,1,0,7.2500,0,0,1,0,0,1
2,1,1,38.0,1,0,71.2833,1,0,0,1,0,0
3,1,1,26.0,0,0,7.9250,0,0,1,0,0,1
4,1,1,35.0,1,0,53.1000,1,0,0,0,0,1
5,0,0,35.0,0,0,8.0500,0,0,1,0,0,1


In [11]:
def get_metrics(y_test, y_score, b=1):
    metrics = {}
    
    precision, recall, thresholds = precision_recall_curve(y_test.values, y_score)
    fscore = (1+b**2)*(precision * recall) / (b**2*precision + recall)
    ix = np.argmax(fscore)
    
    metrics['threshold'] = thresholds[ix]
    metrics['fscore'] = fscore[ix]
    metrics['precision'] = precision[ix]
    metrics['recall'] = recall[ix]
    metrics['roc_auc'] = roc_auc_score(y_test, y_score)
    return metrics

def run_classifier(clf, X_train, X_test, y_train, y_test):
    clf.fit(X_train, y_train)
    y_score = clf.predict_proba(X_test)[:, 1]
    return y_score, get_metrics(y_test, y_score)

In [12]:
X_train, X_test, y_train, y_test = train_test_split(df.drop('Survived', axis=1),
                                                    df['Survived'], random_state=0)

In [13]:
proba, metrics = run_classifier(RandomForestClassifier(max_depth=7, random_state=42), X_train, X_test, y_train, y_test)
metrics

{'threshold': 0.2999806867574714,
 'fscore': 0.8,
 'precision': 0.7169811320754716,
 'recall': 0.9047619047619048,
 'roc_auc': 0.8964114422747516}

In [14]:
# разделить набор данных на два множества: P (positives) и U (unlabeled). 
# Причем брать нужно не все положительные (класс 1) примеры, а только лишь часть

df2 = df.copy()
df2['Survived_orig'] = df2['Survived'] 
df2['Survived'] = df2['Survived'].apply(lambda x: int(np.random.random_sample() < 0.66) if x==1 else 0, 1) # 30% от P отправляем в U

print(df['Survived'].value_counts())
print(df2['Survived'].value_counts())
print(df2['Survived'].value_counts()[1]/df['Survived'].value_counts()[1])

0    549
1    342
Name: Survived, dtype: int64
0    672
1    219
Name: Survived, dtype: int64
0.6403508771929824


In [15]:
# для применения random negative samplin разделим данные на P, U

P = df2[df2['Survived'] == 1]
U = df2[df2['Survived'] == 0]

# выделим из U множество N случайным образом

N = U.sample(frac=0.6) # 60% от U отправляем в N

N.shape[0]/U.shape[0]

0.5997023809523809

In [16]:
X_train = pd.concat([P, N]).sample(frac=1)
y_train = X_train['Survived']
X_train.drop(['Survived', 'Survived_orig'], axis=1, inplace=True)

X_test = U[~U.index.isin(N.index)]
y_test = X_test['Survived_orig']
X_test.drop(['Survived', 'Survived_orig'], axis=1, inplace=True)

c:\users\homeuser\appdata\local\programs\python\python39\lib\site-packages\pandas\core\frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [17]:
proba, metrics = run_classifier(RandomForestClassifier(max_depth=7, random_state=42), X_train, X_test, y_train, y_test)
metrics

{'threshold': 0.514784533850926,
 'fscore': 0.6582278481012658,
 'precision': 0.7878787878787878,
 'recall': 0.5652173913043478,
 'roc_auc': 0.8255020471826867}

Вывод: точность прогноза в случае с Unlabeled данными уменьшилась, так как уменьшилось качество обучающей выборки - некоторые данные ошибочно отнесены к группе 0